# YamboConvergence: automated BSE convergence with respect to the k-points density

In the following, we will perform BSE convergence using the same workflow as for the GW simulations: `YamboConvergence`.
The purpose is to compute BSE convergence with respect to the k-points density, instead of pre-defined k-points meshes.

In [1]:
from aiida import orm, load_profile
load_profile()

from aiida.plugins import WorkflowFactory
from aiida.orm import QueryBuilder

from aiida_quantumespresso.common.types import ElectronicType

import yaml


qb = QueryBuilder()
qb.append(orm.Group, filters={'label': 'hBN/bulk'}, tag='group')
qb.append(orm.StructureData, with_group='group')

loaded_structure_id = qb.all()[0][0].pk

# Read YAML file
with open("../configuration/codes_localhost.yaml", 'r') as stream:
    codes = yaml.safe_load(stream)
    
with open("../configuration/resources_localhost.yaml", 'r') as stream:
    resources = yaml.safe_load(stream)
    
options = {
    'pseudo_family':"PseudoDojo/0.4/PBE/SR/standard/upf",
    'protocol':'fast',
    #'parent_id':274, #not necessary to set; if you want it, take ytheour previously nscf id (pk) to skip the DFT part.
    'structure_id':loaded_structure_id,
}
options.update(codes)

overrides_scf = {
        'pseudo_family': "PseudoDojo/0.4/PBE/SR/standard/upf", 
        'pw':{
            
        'metadata':resources,
        },
    }

overrides_nscf = {
        'pseudo_family': "PseudoDojo/0.4/PBE/SR/standard/upf", 
        'pw': {
            'parameters':{
                'CONTROL':{}, #not needed if you don't override something
                'SYSTEM':{},
                'ELECTRONS':{'diagonalization':'cg'},
            },
             'metadata':resources,
    },
}

overrides_yambo = {
        "yambo": {
            "parameters": {
                "arguments": [
                    "rim_cut",
                ],
                "variables": {
                    "FFTGvecs": [20, "Ry"],
                    'GbndRnge': [[1, 50], ''],
                    'BndsRnXs': [[1, 50], ''],
                    'NGsBlkXs': [2, 'Ry'],
                    'BSENGBlk': [2, 'Ry'],
                    'KfnQP_E':[[1.5,1,1],''],           # <== Scissor and stretching correction.
                    'BS_CPU':str(int(16/2))+' 2 1',     # <== PARALLELISM INFO
                    'BS_ROLEs':'k eh t',                # <== PARALLELISM INFO
                },
            },
        'metadata':resources,
        },
    
}

#Be careful with the mesh choice!!! 
overrides_meta = {
        'FFTGvecs': {
            'start_ratio': 0.25,
            'stop_ratio': 0.7,
            'delta_ratio': 0.1,
            'max_ratio': 1,
        },
        'bands': {
            'start': 50,
            'stop': 400,
            'delta': 50,
            'max': 600,
            'ratio':[10,25,50],
        },
        'G_vectors': {
            'start': 2,
            'stop': 8,
            'delta': 1,
            'max': 10,
        },
        'kpoint_density': {
            'start': 1,
            'stop': 0.4,
            'delta': 1,
            'max': 0.1,
        } ,
        'conv_thr_k': 5,   # <== 5% for convergence wrt super-converged estimation.
        'conv_thr_bG': 10,
        'conv_thr_FFT': 10,
        'conv_thr_units': '%', # 'eV'

        
    }

        
overrides_wfl_settings = {
        'what':['lowest_exciton'],          # <== converging the lowest exciton.
        'type': 'heavy', #or cheap; heavy uses converged value for parameters that we are not converging in a given iteration.
    }

overrides = {
        'ywfl':{'scf':overrides_scf,'nscf':overrides_nscf,'yres':overrides_yambo},
        'workflow_settings':overrides_wfl_settings,
    }


YamboConvergence = WorkflowFactory('yambo.yambo.yamboconvergence')
builder = YamboConvergence.get_builder_from_protocol(
    pw_code = options['pwcode_id'],
    preprocessing_code = options['yamboprecode_id'],
    code = options['yambocode_id'],
    protocol=options['protocol'],
    protocol_qe=options['protocol'],
    structure=orm.load_node(options['structure_id']),
    overrides=overrides,
    #parent_folder=load_node(options['parent_id']).outputs.remote_folder,
    electronic_type=ElectronicType.INSULATOR, #default is METAL: smearing is used
    calc_type='bse', #or 'bse'; default is 'gw'
)

builder.ywfl.scf.pw.metadata.options = resources

builder.ywfl.nscf.pw.metadata.options = builder.ywfl.scf.pw.metadata.options
builder.ywfl.yres.yambo.metadata.options = builder.ywfl.scf.pw.metadata.options

#group
try:
    g = orm.load_group('tutorial/hBN/convergence/BSE')
except:
    g = orm.Group('tutorial/hBN/convergence/BSE')
    g.store()
    
builder.group_label = orm.Str('tutorial/hBN/convergence/BSE') # verdi group create tutorial/hBN/convergence; all calculationsc are added to the group


# parameters:
builder.ywfl.yres.yambo.parameters = orm.Dict(
    dict={'arguments':['em1s','bse','bss','optics', 'dipoles',],
                'variables':{
                'BSEmod': 'resonant',
                'BSKmod': 'SEX',
                'BSSmod': 'd',
                'Lkind': 'full',
                'NGsBlkXs': [2, 'Ry'],
                'BSENGBlk': [2, 'Ry'],
                'Chimod': 'hartree',
                'DysSolver': 'n',
                'BEnSteps': [10,''],
                'BSEQptR': [[1,1],''],
                'BSEBands': [[8,9],''],
                'BEnRange': [[0.0, 10.0],'eV'],
                'BDmRange': [[0.1, 0.1],'eV'],
                'BLongDir': [[1.0, 1.0, 1.0],''],
                'LongDrXp': [[1.0, 1.0, 1.0],''],
                'LongDrXd': [[1.0, 1.0, 1.0],''],
                'LongDrXs': [[1.0, 1.0, 1.0],''],
                'BndsRnXs': [[1,50], ''],
                'KfnQP_E':[[1.5,1,1],''],
                'BS_CPU':str(int(16/2))+' 2 1',
                'BS_ROLEs':'k eh t',
                },}
)

Profile<uuid='3745313b017b418697d37cbf16c4e7ef' name='generic'>

## Solution --> k-points density:

In [ ]:
builder.parameters_space = orm.List(list=[
 {'var': ['kpoint_density'],
  'start': 1/0.8, # Angstrom, or 1/(max distance in reciprocal space, in 1/A)
  'stop': 1/0.4,
  'delta': 0.5,   
  'max': 1/0.2,
  'steps': 4,
  'max_iterations': 4,
  'conv_thr': 3,
  'conv_thr_units': '%',
  'convergence_algorithm': 'new_algorithm_1D'},])

In [52]:
run = None

In [53]:
if run:
    print('run is already running -> {}'.format(run.pk))
    print('sure that you want to run again?, if so, copy the else instruction in the cell below and run!')
else:
    run = submit(builder)

print(run)

/home/max/.conda/envs/aiida/lib/python3.9/site-packages/aiida_quantumespresso/calculations/pw.py:211: UserWarning: `parent_folder` not provided for `nscf` calculation. For work chains wrapping this calculation, you can disable this warning by excluding the `parent_folder` when exposing the inputs of the `PwCalculation`.
  warnings.warn(


uuid: be5a7185-de39-4f68-80c1-813c37dd608f (pk: 1401) (aiida.workflows:yambo.yambo.yamboconvergence)


# Output analysis.

suppose that your calculation completed successfully, then you can access the outputs via the output method of the run instance: 

In [54]:
run.is_finished_ok

False

In [58]:
!verdi process report {run.pk}

2024-01-16 16:23:52 [643 | REPORT]: [1401|YamboConvergence|start_workflow]: group: tutorial/hBN/convergence/BSE
2024-01-16 16:23:52 [644 | REPORT]: [1401|YamboConvergence|start_workflow]: Workflow type: heavy; looking for convergence of ['lowest_exciton']
2024-01-16 16:23:52 [645 | REPORT]: [1401|YamboConvergence|start_workflow]: Workflow initilization step completed, the parameters will be: ['kpoint_mesh'].
2024-01-16 16:23:52 [646 | REPORT]: [1401|YamboConvergence|has_to_continue]: Still iteration on ['kpoint_mesh']
2024-01-16 16:23:52 [647 | REPORT]: [1401|YamboConvergence|pre_needed]: {'kpoint_mesh': [[4, 4, 2], [5, 5, 3], [7, 7, 3], [8, 8, 4]]}
2024-01-16 16:23:52 [648 | REPORT]: [1401|YamboConvergence|next_step]: New parameters are: {'kpoint_mesh': [4, 4, 2]}
2024-01-16 16:23:52 [649 | REPORT]: [1401|YamboConvergence|next_step]: Calculation already done: 1244
2024-01-16 16:23:54 [650 | REPORT]: [1401|YamboConvergence|next_step]: New parameters are: {'kpoint_mesh': [5, 5, 3]}
2024

The converged parameters can be obtained via the "infos" output Dict:

In [134]:
run.outputs.infos.get_dict()

{'E_ref': 5.6156411578646,
 'kpoint_mesh': [7, 7, 4],
 'extrapolation': 5.6175120483652,
 'lowest_exciton': 5.5455183982849}

The full convergence history can be visualized in a table form using pandas:

In [135]:
import pandas as pd

In [136]:
history = run.outputs.history.get_dict()

In [137]:
history_table = pd.DataFrame(history)

In [138]:
history_table

,uuid,failed,useful,global_step,kpoint_mesh,lowest_exciton,parameters_studied
0,1b01b60f-bf26-4c4a-be35-d145547e635c,False,False,1,"[4, 4, 2]",5.359427,kpoint_mesh
1,37fcb1a7-b998-44d6-8c57-89facc0f4f90,False,True,2,"[7, 7, 2]",5.545518,kpoint_mesh
2,a0414f3e-09c3-47e8-b573-830d2f776c71,False,False,3,"[13, 13, 5]",5.598341,kpoint_mesh
3,3aeb473e-6747-4397-8802-c4cdbcfba2a3,False,False,4,"[16, 16, 6]",5.623181,kpoint_mesh
4,e1fc6ebf-06f6-40f4-ac99-38d6375e473a,False,False,5,"[7, 7, 4]",5.471172,kpoint_mesh


The last calculations can be obtained using:

In [139]:
history_table[history_table['useful']==True]

,uuid,failed,useful,global_step,kpoint_mesh,lowest_exciton,parameters_studied
1,37fcb1a7-b998-44d6-8c57-89facc0f4f90,False,True,2,"[7, 7, 2]",5.545518,kpoint_mesh


Result on the convergence path can be plotted using several plotting libraries, for examples here we are gonna use [plotly]((https://plotly.com/python/3d-line-plots/)) to observe the convergence between bands and plane wave cutoff for the screening matrix:

In [143]:
import plotly.express as px

k_history = history_table[history_table['parameters_studied']=="kpoint_mesh"]

df = k_history
fig = px.scatter(df, y="lowest_exciton", text=df["kpoint_mesh"])
fig.show()

## Exercise 1: try to compute BSE convergence but using QP instead of scissor and stretching.

Solution provided in the `Solution_2_YamboConvergence_BSE_QP.ipynb` notebook. 
Please look at the `6_1_YamboWorkflow_BSE_QP.ipynb` to understand how to set the preliminary QP calculation.

## Exercise 2: computing the convergence not with respect to the k-mesh, but with respect to the k-point density.

Solution provided in the `Solution_3_YamboConvergence_BSE_k_density.ipynb` notebook.

```python
builder.parameters_space = orm.List(list=[
 {'var': ['kpoint_density'],
  'start': 1/0.05,
  'stop': 1/0.025,
  'delta': 1,      #1/0.01
  'max': 1/0.02,
  'steps': 4,
  'max_iterations': 4,
  'conv_thr': 3,
  'conv_thr_units': '%',
  'convergence_algorithm': 'new_algorithm_1D'},])
```